In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("employee_data.csv")

In [3]:
df.fillna('',inplace=True)

In [4]:
df.drop_duplicates('full_name',inplace=True, ignore_index=True)

In [5]:
df.date_hired = pd.to_datetime(df.date_hired, utc=True)

In [6]:
df.sort_values(['date_hired','full_name'],inplace=True)
df.reset_index(drop=True,inplace=True)
df['SN'] = df.index + 1
df['unique_id'] = 'BM-' + df['SN'].astype('str') + '-US'
print(df.unique_id.duplicated().sum())

0


In [7]:
df.duplicated('full_name').sum()

0

In [8]:
df.duplicated('email_address').sum()

0

In [9]:
df.shape

(995, 178)

In [10]:
df.duplicated('unique_id').sum()

0

In [11]:
df.date_hired.isna().sum()

0

In [12]:
df.sort_values(['date_hired','full_name'],inplace=True)

In [13]:
df.columns

Index(['first_name', 'last_name', 'full_name', 'phone_number', 'email_address',
       'SN', 'region', 'unique_id', 'date_hired', 'address',
       ...
       'checkin_location_10_Dec_23', '_17_Dec_23', 'comment17122023',
       'checkin_location_17_Dec_23', '_24_Dec_23', 'comment24122023',
       'checkin_location_24_Dec_23', '_31_Dec_23', 'comment31122023',
       'checkin_location_31_Dec_23'],
      dtype='object', length=178)

In [14]:
df.columns = df.columns.str.replace('checkin','input')

In [15]:
df.columns

Index(['first_name', 'last_name', 'full_name', 'phone_number', 'email_address',
       'SN', 'region', 'unique_id', 'date_hired', 'address',
       ...
       'input_location_10_Dec_23', '_17_Dec_23', 'comment17122023',
       'input_location_17_Dec_23', '_24_Dec_23', 'comment24122023',
       'input_location_24_Dec_23', '_31_Dec_23', 'comment31122023',
       'input_location_31_Dec_23'],
      dtype='object', length=178)

In [16]:
df.rename(lambda x: x.split('_')[0] + "-" + x.split('_')[1] + "-" + str(int(x.split('_')[2])-2).zfill(2) +
          "-" + x.split('_')[3] + "-" + x.split('_')[4]
          if any(k in x for k in ['input_location_']) else x, axis=1, inplace=True)

In [17]:
df.rename(lambda x:  "_" + str(int(x.split('_')[1])-2).zfill(2) +
          "_" + x.split('_')[2] + "_" + x.split('_')[3]
          if any(k in x for k in ['_23']) else x, axis=1,inplace=True)

In [18]:
df.rename(lambda x: x[:7] + str(int(x[7:9])-2).zfill(2) + x[9:]
          if any(k in x for k in ['comment']) else x, axis=1,inplace=True)

In [19]:
df.columns = df.columns.str.replace('-','_')

In [29]:
df.columns = df.columns.str.replace('00_Apr','31_Mar')
df.columns = df.columns.str.replace('00042023','31032023')
df.columns = df.columns.str.replace('00_Jul','30_Jun')
df.columns = df.columns.str.replace('00072023','30062023')
df.columns = df.columns.str.replace('_1_Oct','29_Sep')
df.columns = df.columns.str.replace('_1102023','29092023')

In [30]:
df.columns[120:180]

Index(['input_location_18_Aug_23', '_25_Aug_23', 'comment25082023',
       'input_location_25_Aug_23', '_01_Sep_23', 'comment01092023',
       'input_location_01_Sep_23', '_08_Sep_23', 'comment08092023',
       'input_location_08_Sep_23', '_15_Sep_23', 'comment15092023',
       'input_location_15_Sep_23', '_22_Sep_23', 'comment22092023',
       'input_location_22_Sep_23', '_29_Sep_23', 'comment29092023',
       'input_location_29_Sep_23', '_06_Oct_23', 'comment06102023',
       'input_location_06_Oct_23', '_13_Oct_23', 'comment13102023',
       'input_location_13_Oct_23', '_20_Oct_23', 'comment20102023',
       'input_location_20_Oct_23', '_27_Oct_23', 'comment27102023',
       'input_location_27_Oct_23', '_03_Nov_23', 'comment03112023',
       'input_location_03_Nov_23', '_10_Nov_23', 'comment10112023',
       'input_location_10_Nov_23', '_17_Nov_23', 'comment17112023',
       'input_location_17_Nov_23', '_24_Nov_23', 'comment24112023',
       'input_location_24_Nov_23', '_01_Dec_23',

In [39]:
def recalc_att_ytd(dataframe, today): # select sundays on or before sundays
    date_list_column = [] # create list to store dates from date columns
    start_column = dataframe.columns.get_loc('_06_Jan_23') # select index of first date column
    while int(start_column) < dataframe.shape[1]: # store date columns into the list
        date_list_column.append(pd.to_datetime(dataframe.columns[start_column], format='_%d_%b_%y', utc=True))
        start_column = int(start_column) + 3
    for i in dataframe.index: # for every person in the table
        date_hired = dataframe.loc[i,'date_hired'] # select date added
        dates_available = [] # create list to store all dates in the table that are after when the user was added to the table and are on or before today
        for date in date_list_column: # for every date present in the original table
            if date >= date_hired and date <= pd.to_datetime(today, utc=True): # check if the date is on or after the user was added and on or before today
                column_name = date.strftime('_%d_%b_%y')
                dates_available.append(column_name) # store that date into a list
        sundays_by_today = dataframe.loc[i,dates_available] # create dataframe with only those date columns
        sundays_by_today.fillna('',inplace=True)
        all_count = sundays_by_today.count() # count total days
        try:
            present_count = sundays_by_today.value_counts()['Present'] # count days present
            dataframe.loc[i,'att_ytd'] = round((present_count/all_count*100),2) # recalculate att_ytd
        except: dataframe.loc[i,'att_ytd'] = 0.00
    return dataframe

In [34]:
from datetime import date
today = date.today()

In [37]:
df.date_hired = pd.to_datetime(df.date_hired, utc=True)

In [40]:
df = recalc_att_ytd(df,today)

In [41]:
df.att_ytd.value_counts(dropna=False)

50.00     111
55.56      80
44.44      65
38.89      59
61.11      56
         ... 
100.00      1
61.54       1
26.67       1
38.46       1
18.18       1
Name: att_ytd, Length: 77, dtype: int64

In [42]:
df.dtypes

first_name                  object
last_name                   object
full_name                   object
phone_number                 int64
email_address               object
                             ...  
comment22122023             object
input_location_22_Dec_23    object
_29_Dec_23                  object
comment29122023             object
input_location_29_Dec_23    object
Length: 178, dtype: object

In [44]:
df = df.replace('Absent','Incomplete')

In [45]:
df = df.replace('Present','Completed')

In [46]:
df.to_csv('employee_data.csv',index=False)